# Laborator 7 - Model Exercitii Colocviu

## Imports

In [6]:
import numpy as np
from urllib import request
import gzip
import pickle
from tqdm import tqdm

## Data Loading

- Datele pe care le vom folosi astazi se gasesc intr-un fisier "data.pkl", care contine setul de date MNIST sub forma de dictionar Python "pickle".

De aici luam imaginile si label-urile de antrenare si de testare.

In [7]:
def load():
    with open("data.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

In [8]:
train_images, train_labels, test_images, test_labels = load()

- Imaginile sunt sub forma de vector pickle, asa ca am vrea sa le formatam inapoi la forma lor originala, o matrice de dimensiunea (N, 28, 28), unde N = numarul de samples.

- Daca folosim .reshape() cu primul argument egal cu "-1", numpy deduce automat dimensiunea (numarul de imagini).

Pasul asta e esential ca sa procesam vecinitatile, din moment ce ele au sens doar in (minim) 2D.

In [9]:
train_images = train_images.reshape(-1, 28, 28)#[:5000]
test_images = test_images.reshape(-1, 28, 28)#[:2000]

In [10]:
train_labels = train_labels#[:5000]
test_labels = test_labels#[:2000]

- Putem transforma prima imagine intr-un obiect de tip "Image" ca sa o vizualizam:

'astype(np.uint8)' e necesar pentru ca valorile de intensitate sa fie valide intre 0 si 255.

(Pasul acesta nu e esential pentru colocviu, e util doar ca sa intelegem/vizualizam datele)

In [11]:
from PIL import Image
Image.fromarray(train_images[0].astype(np.uint8))

## Exercitiul 1

- Avem nevoie de o functie ajutatoare, care sa ne transforme un vector binar (de ex. [1, 0, 1] intr-un numar in baza 10).

- In plus, folosim reversed() ca sa punem cel mai putin semnificativ bit la dreapta.

- Transformarea (scalarul) o sa fie indexul nostru unic pentru histograma de vectori.

Spre exemplu, pentru un patch de 3x3:

[[60, 70, 55],

 [58, 65, 62],
 
 [50, 68, 59]]

 Pixelul central e "65", pe care il comparam cu ceilalti 8 si obtinem binary signature-ul: "01000010" -> [0, 1, 0, 0, 0, 0, 1, 0]

 Deci, binary2decimal o sa imi returneze: $results = 0 \cdot 2^7 + 1 \cdot 2^6 + ... + 1 \cdot 2^1 + 0 \cdot 2^0 = 64 + 2 = 66.$

 Asadar, pozitia 66 din histograma se incrementeaza cu o frecventa de 1.

In [ ]:
def binary2decimal(binary):
    return sum(val*(2**idx) for idx, val in enumerate(reversed(binary)))

In [90]:
def f_ex1(image, d=3):

    # Calculam cat ar trebui sa luam din fiecare directie ca sa pastram centrul vecinatatii pixelului/patch-ului in interiorul imaginii.
    h, w = image.shape
    size_left_right = (d-1)//2 # Pentru d=3, o sa avem 1 pixel la stanga, 1 la dreapta, 1 sus, 1 jos. 
    size_up_down = (d-1)//2

    # Adaugam un padding cu "-1" ca sa putem calcula vecinatatile pixelilor din marginile imaginii 
    # "-1" nu influenteaza calculele, deoarece orice pixel real are valoare >=0.
    padded_image = np.pad(image, ((size_up_down, size_up_down), (size_left_right, size_left_right)), constant_values=-1) # (28, 28) -> (30, 30)
    # Alternativ, pentru cazul de fereastra de 3x3: padded_image = np.pad(image, ((1,1), (1,1)), constant_values=-1)
    
    # Iteram peste fiecare pixel care are o vecinatate complet definita (evitam intai marginile) 
    # Vrem sa extragem acel binary signature pt fiecare pixel (cu vecinatatea aferenta)
    results = []
    for i in range(size_up_down, h):
        for j in range(size_left_right, w):

            # Initializam un vector de output (d x d) care o sa contina toate valorile binare din comparatia pixelului cu "vecinii" sai.
            output_representation = np.zeros((d*d)-1)
            # Luam un patch de dimensiune tot d x d (3x3 in cazul nostru) centrata pe pixelul curent din iteratie
            patch = padded_image[i-size_left_right:i+size_left_right+1,
                                 j-size_up_down:j+size_up_down+1]

            # Valoare de comparat este chiar valoarea pixelului de la pozitia (i, j)
            value = padded_image[i][j]
            # Comparam fiecare pixel din patch cu valoarea centrala
            # "*1" converteste True/False in 1 / 0.
            comparison = (patch>value)*1
            # Vectorizam patch-ul de comparatii intr-un vector 1D de lungime d*d.
            result = comparison.reshape(-1)

            # Scoatem valoarea centrala din patch-ul vectorizat (nu vrem sa comparam pixelul cu el insusi)
            result = np.delete(result, (d*d)//2)
            results.append(result)
            
            # output_representation[:len(output_representation)//2] = result[:len(output_representation)//2]
            # output_representation[len(output_representation)//2:] = result[len(output_representation)//2 + 1:]
            # results.append(output_representation)

    # Initializam o histograma (vector) de lungime 2^(d*d-1)
    # In cazul nostru 2^8 combinatii posibile de biti = 256 de valori/semnaturi diferite pentru pixeli (0-255)
    # Histograma e in esenta o reprezentare/semnatura globala a imaginii
    # Practic, avem 784 pixeli per imagine -> 784 de vectori/"semnaturi binare" -> le agregam count-ul intr-o histograma de frecvente (256 valori posibile)
    histogram = np.zeros((2**(d*d-1))) # Histograma = vector/array de frecvente
    # 2 ^ 8 = 256
    for result in results:
        position = binary2decimal(result)
        histogram[int(position)]+=1
    return histogram

def process_data(images):
    results = []
    # Aplicam f_ex1 pentru fiecare imagine din setul de dat (tqdm ne ajuta pentru progres vizual -> Nu e obligatoriu, dar ajuta la debugging)
    for image in tqdm(images):
        hist = f_ex1(image)
        results.append(hist)
    # Obtinem un vector de features pentru tot setul de imagini pe care aplicam functia
    return np.array(results)

In [ ]:
train_hist = process_data(train_images) # 256 x 5000
test_hist = process_data(test_images) # 256 x 2000

In [103]:
train_hist[0].shape

(256,)

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# Importam un scaler pentru normalizare si normalizam histogramele (evitam dominanta unor frecvente mari)
scaler = StandardScaler()
scaler.fit(train_hist)
train_hist_scaled = scaler.transform(train_hist)
test_hist_scaled = scaler.transform(test_hist)

# Antrenam modelul SVM pe histograma normalizata
clf = SVC(C = 10, kernel = 'rbf', gamma = 0.001)
clf.fit(train_hist_scaled, train_labels)

In [111]:
# Calculam acuratetea pe setul de test
print("Accuracy:", clf.score(test_hist_scaled, test_labels) * 100, "%")

Accuracy: 77.75 %


### De ce am obtinut atat?

O acuratete de 77.75% e acceptabila pentru o metoda facuta "de mana".

Avantaje:
- Nu am folosit pixelii bruti, ci niste reprezentari bazate pe semnaturi binare (similar cu LBP - Local Binary Patterns)
- Vectorul de features cu care lucram are 256 dimensiuni per imagine, deci datele sunt comprimate, iar SVM lucreaza bine eficient
- Nu ne folosim de DL sau retele convolutionale, e doar ML clasic cu prelucrare de date

Dezavantaje:
- Binary signature-ul calculat de noi pierde informatii, deoarece:
  - Nu tine cont de valoarea absoluta, doar de "mai mare" sau "mai mic" decat pixelul centrat
  - Nu invata formele sau muchiile complexe, pentru ca histograma sterge informatia de pozitie. Fiecare vector binar e agregat global, ceea ce inseamna ca de exemplu Imaginile cu "2" si cele cu "5" pot avea parti comune ca binary signatures, adica o histograma similara.

## Exercitiul 2

In esenta, vrem sa calculam magnitudinea gradientului unei imagini, sa o impartim in regiuni de 3x3, dupa care sa selectam cele mai "informative" k regiuni (care au gradient mediu mare), si sa le folosim pentru antrenarea unui model ML.

- Gradientul ne ofera o masura locala a variatiei intensitatii; E util pentru a identificare contururi, margini, regiuni mai informative.

In [1]:
import cv2

# Calculam magnitudinea gradientului unei imagini folosind filtrul Sobel
# Sobel = filtru (kernel) folosit in CV pentru a estima gradientul intensitatii intr-o imagine. 
# Practic, detecteaza margini, contururi si zone unde apar modificari bruste in imagine (luminozitatea locala)
# E o aproximare discreta a primelor derivate si foloseste doua matrici de convolutie (pentru Ox si Oy).
def compute_gradient_magnitude(image):
    # Aplica filtrul Sobel pe directia Ox (orizontala)
    gx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3) # CV_64F = precizie ridicata, evita pierderea semnelor negative 
                                                         # ksize = 3 -> kernel Sobel de (3x3)
    # Aplica filtrul Sobel pe directia Oy (verticala)
    gy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3) 

    # Formula exacta pentru magnitudinea gradientului
    grad_mag = np.sqrt(np.square(gx) + np.square(gy))  # Cat de "puternic" e conturul
    return grad_mag # grad_mag.shape == (28, 28) | Fiecare pixel contine cat de "abrupt" se schimba intensitatea in zona aceea.

# Pentru fiecare imagine, calculam gradientul (tqdm doar adaug un progress bar)
def process_images(images):
    results = []
    for image in tqdm(images):
        results.append(compute_gradient_magnitude(image))
    return np.array(results)

In [41]:
# Calculam matricea de gradienti pentru seturile de antrenare si testare
train_grad_mag = process_images(train_images)
test_grad_mag = process_images(test_images)

100%|██████████| 2000/2000 [00:00<00:00, 39318.53it/s]


In [31]:
# In rezultatul final, train_grad_mag[i] are aceleasi dimensiuni ca train_images[i], dar valorile sunt magnitudini de gradient.
print("train_images shape:", train_images.shape)
print("train_grad_mag shape:", train_grad_mag.shape)

train_images shape: (5000, 28, 28)
train_grad_mag shape: (5000, 28, 28)


In [32]:
# Selectam cele mai informative "k" regiuni de dimensiuni (d, d) din imaginea "original_image", careia ii corespunde gradientul "grad_mag"
def get_top_k_regions(grad_mag, original_image, k=30, d=4): #30/49
    # Presupunem ca imaginea are dimensiuni divizibile exact cu "d"
    # In cazul nostru, 28x28 = 784 pixeli -> pentru d=4, o sa avem 7x7 patch-uri de 4x4

    # Transformam imaginea intr-un vector de patch-uri de dimensiuni (d, d). Numpy stie singur ca o sa obtina 49 de patch-uri, de aceea punem "-1".
    patches_mag = grad_mag.reshape(-1, d, d) # patches_mag.shape == (49, 4, 4)
    original_image = original_image.reshape(-1, d, d)

    # Calculam media gradientului in fiecare patch => masura noastra pentru complexitate / "informatie" / scor de importanta vizuala per patch
    patches_mag = np.mean(patches_mag, axis=(1, 2))  # patches_mag.shape == (49,)

    # Sortam patch-urile in functie de relevanta (gradientii slabi -> sus, iar cei puternici -> jos)
    sorted_indices = np.argsort(patches_mag)

    # Pastram doar patch-uri originale corespunzatoare celor mai "puternice" k regiuni din gradient
    output = original_image[sorted_indices[-k:]]
 
    # Concatenam toate patch-uri selectate intr-un singur vector
    return np.array(output).reshape(-1)

# Pentru fiecare imagine, extragem patch-urile informative si le transformam intr-un vector
# NU PASTRAM VALORILE DE GRADIENT ALE PATCH-URILOR! Ci patch-urile brute din imaginea originala, dar doar cele cu media gradientului cea mai mare.
def transform_images(images, grad_mags):
    results = []
    for i, image in enumerate(tqdm(images)):
        output = get_top_k_regions(grad_mags[i], image)
        results.append(output)
    return np.array(results)

In [ ]:
ex2_train_data = transform_images(train_images, train_grad_mag)
ex2_test_data = transform_images(test_images, test_grad_mag)

100%|██████████| 2000/2000 [00:00<00:00, 38947.04it/s]


In [34]:
# Rezultatul final ex2_train_data are dimensiunea (N, k*d*d) = (5000, 30* 4*4) => Fiecare imagine devine un vector de features de dimensiune fixa
# 5000 de imagini, 30 de patch-uri pastrate, 16 valori per patch. Practic extragem cei mai relevanti 480 pixeli din 784.
print("ex2_train_data shape:", ex2_train_data.shape)
print("ex2_test_data shape:", ex2_test_data.shape)

ex2_train_data shape: (5000, 480)
ex2_test_data shape: (2000, 480)


In [39]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(ex2_train_data)
ex2_train_data_scaled = scaler.transform(ex2_train_data)
ex2_test_data_scaled = scaler.transform(ex2_test_data)

clf = SVC(C=10)
clf.fit(ex2_train_data_scaled, train_labels)

SVC(C=10)

In [38]:
print("Accuracy:", clf.score(ex2_test_data_scaled, test_labels) * 100, "%")

Accuracy: 47.05 %


### De ce am obtinut mai bine decat la Ex. 1?

Fata de exercitiul anterior, care era bazat pe comparatii locale si pierdea informatiile de pozitie, acum am pastrat mai multa informatie vizuala utila:

- Am ales doar zonele cu variatie locala mare (margini, forme), care sunt mai reprezentative pentru cifre
- Am pastrat valorile reale brute ale pixelilor, nu doar relatii binare
- Chiar daca nu avem o structura spatiala, cele mai informative patch-uri tin sa apara in zone cheie ale cifrelor (mijloc, bucle etc.), deci patch-urile pastreaza o "urma" de pozitie

## Exercitiul 3

Similar cu exercitiul anterior, vrem sa calculam magnitudinea si directia gradientului.

Dupa care, pentru fiecare pixel, comparam magnitudinea gradientului sau cu a celor doi vecini de pe directia gradientului (Ox sau Oy). Daca e mai mare decat amandoi, pastram valoarea, altfel punem 0.

Mai exact:
- Calculam gradientul de intensitate pe directiile x si y (sobelx, sobely)
- Obtinem magnitudinea gradientului (cat de puternica e variatia) si directia acestuia
- Facem NMS - comparam fiecare pixel cu vecinii de pe directia gradientului (pastram doar maximele locale)
- Obtinem o imagine trecuta prin NMS (non-maximum suppression), noua, tot de dimensiune 28x28, in care:
  - Doar marginile semnificative au valori nenule
  - Restul pixelilor sunt 0

Gradientul intr-o imagine e un vector care ne spune

"Cat de repede si in ce directie se schimba intensitatea pixelilor in vecinatatea unui punct?"

In [41]:
# Calculam magnitudinea fiecarui gradient identic cu exercitiul anterior, folosint filtrul Sobel
# Parametrii in plus "1" si "0" reprezinta derivata pe x si y (pe directie: dx, dy)
# In esenta Gx si Gy sunt componentele gradientului intr-o imagine (pentru ca gradientul variaza in 2 directii => imaginea e o matrice 2D)
def compute_gradient_direction(image):
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3) # Gx
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3) # Gy
    # Calculam unghiul directiei gradientului in radiani
    grad_direction = np.arctan2(sobely, sobelx) # In ce directie "urca" intensitatea
    # Folosim artcan2 si nu arctan ca sa tinem cont de semnele lui Gx si Gy
    # => unghiul corect in toate cadranele (ex. pentru Gx = -1 si Gy = 1, ar trebui sa avem valoarea de -135 de grade. Dar Gy/Gx = -1 => arctan(-1) = 45*
    # Arctan2 elimina ambiguitatea si e standard in CV ca sa obtinem directii corecte ale gradientului
    return grad_direction

# Aplicam functia pe toate imaginile
def process_directions(images):
    results = []
    for image in tqdm(images):
        results.append(compute_gradient_direction(image))
    return np.array(results)

train_grad_direction = process_directions(train_images)
test_grad_direction = process_directions(test_images)

def non_max_suppression_gradient(img, direction):
    h, w = img.shape
    # Convertim unghiurile din radiani in grade, si ne asiguram ca toate sunt in intervalul [0, 180]
    output = np.zeros((h,w), dtype=np.int32)
    # angle = arctan2(Gy, Gx) e in intervalul [-pi, pi]
    angle = direction * 180. / np.pi # Transformam radianii in grade
    angle[angle < 0] += 180 # Transformam unghiurile in echivalentele lor pozitive (e.g. -45 devine 135, ca sa avem tot intre 0 si 180 grade)

    # Luam vecinii de pe directia gradientului
    # Iteram doar pe interiorul imaginii (evitam marginile)
    for i in range(1, h-1):
        for j in range(1, w-1):
            # Initializam vecinii la 255 ca fallback (valoare mare, deci pixelul probabil nu va fi pastrat)
            neighbour1 = 255
            neighbour2 = 255

            # Impartim in 4 intervale de directie (aproximativ) deoarece NMS presupune ca sa comparam un pixel cu 2 vecini pe directia gradientului
            # Intr-o imagine 2D, deci, o sa avem 8 vecini posibili in jurul unui pixel
            # [0, 22.5)        → directie 0°  
            # [22.5, 67.5)     → directie 45°  
            # [67.5, 112.5)    → directie 90°  
            # [112.5, 157.5)   → directie 135°  
            # [157.5, 180]     → directie 0°

            # Directia 0 - orizontala
            if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                neighbour1 = img[i, j+1] # Dreapta pixelului
                neighbour2 = img[i, j-1] # Stanga pixelului

            # Directia 1 - diagonala 
            elif (22.5 <= angle[i,j] < 67.5):
                neighbour1 = img[i+1, j-1] # Stanga jos de pixel
                neighbour2 = img[i-1, j+1] # Dreapta sus de pixel

            # Directia 2 - verticala
            elif (67.5 <= angle[i,j] < 112.5):
                neighbour1 = img[i+1, j] # Jos (sub pixel)
                neighbour2 = img[i-1, j] # Sus (peste pixel) 

            # Directia 2 - cealalta diagonala
            elif (112.5 <= angle[i,j] < 157.5):
                neighbour1 = img[i-1, j-1] # Stanga sus de pixel
                neighbour2 = img[i+1, j+1] # Dreapta jos de pixel

            # Daca pixelul e mai mare sau egal decat vecinii de pe directia gradientului, pastram valoarea. Altfel, il "suprimam" la 0.
            if (img[i,j] >= neighbour1) and (img[i,j] >= neighbour2):
                output[i,j] = img[i,j]
            else:
                output[i,j] = 0
    
    return output

# Procesam toate imaginile cu functia anterioara
def process_image_with_nms(mags, directions):
    results = []
    for i, mag in enumerate(mags):
        results.append(non_max_suppression_gradient(mag, directions[i]))
        
    return np.array(results)

new_train_mags = process_image_with_nms(train_grad_mag, train_grad_direction)
new_test_mags = process_image_with_nms(test_grad_mag, test_grad_direction)

100%|███████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 36948.69it/s]


SVM se antreneaza pe imaginile procesare dupa NMS:
 - new_train_mags are forma (5000, 28, 28) si trebuie aplatizat la (5000, 784)

Multe din feature-urile finale sunt zero (datorita NMS)

De data aceasta vom folosi alta metoda de scalare, si anume MinMaxScaler, deoarece:
- Datele noastre dupa ce aplicam NMS sunt sparse (majoritatea valorilor sunt 0)
- Doar marginile/contururile au valori semnificative

StandardScaler incearca sa centreze datele la media 0 si deviata standard 1, dar e bun cand avem trasaturi distribuite normal, fara valori sparse. In cazul nostru, media e aproape de 0, deci valorile informative devin prea mici => se pierde contrastul dintre "edge" si "non-edge".

MinMaxScaler aduce toate valorile in intervalul [0, 1] si e bun cand avem date sparse, intensitati etc. El pastreaza proportiile relative si nu strica contrastul dintre noise (in cazul nostru "0") si valoril mai mari.

In [157]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

# Reshape imaginile in formatul (N, 784)
X_train = new_train_mags.reshape(-1, 784)
X_test = new_test_mags.reshape(-1, 784)

# Aplicam scalarea
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = SVC(C=10, kernel = 'rbf')
clf.fit(X_train_scaled, train_labels) 

SVC(C=10)

In [158]:
# Evaluare
score = clf.score(X_test_scaled, test_labels)
print("Test Accuracy:", score * 100, "%")

Test Accuracy: 84.95 %


## Exercitiul 4

Exercitiul 4 presupune urmatoarele lucruri:
- Impartirea fiecarei imagini in patch-uri (similar cu 1 si 2)
- Pentru fiecare patch, sa obtinem un binary signature (ca la ex1, unde facusem comparatiile fata de pixelul central)
- Concatenarea acestor vectori binari -> Avem un vector final per fiecare imagine
- Antrenarea KNN folosind distanta Hamming (numarul de pozitii din doua stringuri care difera)

In [181]:
def binarize_image(image, d=4):
    '''
    Functia binarize_image transforma o imagine intr-un vector binar:
    1. Imparte imaginea in patch-uri d x d (trebuia ca dimensiunea imaginii sa fie divizibila 
    2. Pentru fiecare patch, compara pixelii cu pixelul central (impar) sau media centrala (par)
    3. Rezultatul este un vector de 0/1 per patch, toate concatenate
    '''
    patches = image.reshape(-1, d, d) # Pentru d=4, avem 49 de patch-uri.
    outputs = []
    
    # Iteram prin fiecare patch ca sa il transformam in vector binar
    for patch in patches:
        # Pentru patch-uri de dimensiune para (e.g. 4x4, 6x6 etc.)
        if d % 2==0:
            # NU scoatem pixelul central (nu exista de comparat), deci vectorul binar pastreaza d^2 pixeli.
            output_representation = np.zeros((d*d))
            # NU exista un singur "pixel central" -> luam media din blocul de 2x2 din mijloc
            value = np.mean(patch[d//2 - 1: d//2 + 1, 
                                  d//2 - 1: d//2 + 1])

        # Trebuie sa tratam ambele cazuri, si de patch-uri pare si impare, deoarece aici nu se suprapun si nu mai facem sliding window
        # Pentru patch-uri de dimensiune impara (e.g. 3x3, 5x5, etc.)
        else:
            # Eliminam pixelul central (nu ne comparam cu noi insine), deci vectorul binar are dimensiunea d^2 - 1.
            output_representation = np.zeros((d*d)-1)
            # Pixelul central e cel de la pozitia d//2, d//2 (in cazul unui 3x3 ar fi (1,1) )
            value = patch[d//2, d//2]

        # Comparam fiecare pixel cu valoarea de referinta si producem un array (True/False -> 0/1 cu ajutorul *1)
        comparison = (patch>=value)*1
        # Aplatizam patch-ul
        result = comparison.reshape(-1)

        # In cazul patch-urilor de dimensiune para, nu trebuie sa stergem niciun pixel din output-ul final
        if d % 2==0:
            output_representation = result
        # In cazul impar, similar cu ex 1, eliminam valoarea centrala din vectorul aplatizat
        else:
            output_representation = np.delete(result, d*d // 2)

        outputs.append(output_representation)

    # Obtinem un singur vector binar mare (toate patch-urile concatenate)
    # e.g. pentru o imagine cu 49 de patch-uri de 4x4, rezultatul final o sa aiba 49*16 = 784 biti
    return np.concatenate(outputs)

def process_data(images):
    results = []
    for image in tqdm(images):
        results.append(binarize_image(image))
    return np.array(results)

In [ ]:
train_binary_data = process_data(train_images)
test_binary_data = process_data(test_images)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Aici nu avem nevoie de scaler, deoarece deja avem vectori binari pe care aplicam distanta Hamming
neigh = KNeighborsClassifier(n_neighbors = 7, metric = 'hamming')

neigh.fit(train_binary_data, train_labels)
print("KNN Test Accuracy:", neigh.score(test_binary_data, test_labels)*100, "%")

KNN Test Accuracy: 86.25 %


### De ce am obtinut un rezultat bun?

- Patch-urile pastreaza o oarecare localizare si structura (pentru ca fiecare patch e un mic signature vizual, concatenarea lor mentine ordinea)
- Binarizarea reduce noise-ul (elimina diferentele mici de intensitate intre pixeli)
- Hamming Distance e foarte eficienta pentru a diferentia cati biti difera intre doua imagini

## Exercitiul 5

Vrem sa folosim histogramele calculate la exercitiul 1 ca sa antrenam un SVM cu kernel de intersectie (Intersection Kernel)

### Ce e un Intersection Kernel?

E o functie de similaritate intre doua histograme discrete (vectori pozitivi) si e definita ca

$$ K(h_1, h_2) = \sum_{i=1}^n min(h_1[i], h_2[i]) $$

- Masoara cat de mult se suprapun doua histograme
- Daca doua distributii au valori similare in aceleasi pozitii, scorul va fi mai mare
- E folosit in clasificare de imagini, recunoastere de forme etc

In [218]:
# Definim functia kernelului de intersectie
def intersection_kernel(hist1, hist2):
    return np.sum(np.minimum(hist1, hist2))

# Creem o matrice de similaritate intre doua seturi de histograme
def compute_kernel_matrix(histograms1, histograms2, train=True):
    # histograms1.shape == (5000, 256) | histograms2.shape == (2000, 256)
    kernel_matrix = np.zeros((len(histograms1), len(histograms2)))

    # In cazul de antrenare, construim matricea simetric (optimizare)
    # Comparam fiecare xi cu fiecare xj din setul de antrenare -> Rezultatul are dimensiunea (n_train, n_train) = (5000, 5000) -> matrice simetrica
    if train:
        for i in tqdm(range(len(histograms1))):
            for j in range(i, len(histograms2)):
                similarity = intersection_kernel(histograms1[i], histograms2[j])
                kernel_matrix[i][j] = similarity
                kernel_matrix[j][i] = similarity
    # In cazul de testare, comparam fiecare histograma din setul de test x_i cu fiecare histograma din setul de antrenare x_j
    # Comparam fiecare xi din test cu fiecare xj din train -> Rezultatul are dimensiunea (n_test, n_train) = (2000, 5000) -> asimetrica, nu putem optimiza
    # Scorurile de similaritate obtinute sunt folosite pentru a determina cat de apropiata de imaginea de test fata de clasele invatate
    else:
        for i in tqdm(range(len(histograms1))):
            for j in range(len(histograms2)):
                kernel_matrix[i][j] = intersection_kernel(histograms1[i], histograms2[j])
    return kernel_matrix

Concret, train_matrix[i][j] contine similaritatea (intersectia) dintre histograma imaginii i si histograma imaginii j.

In [221]:
train_matrix = compute_kernel_matrix(train_hist_scaled, train_hist_scaled)
test_matrix = compute_kernel_matrix(test_hist_scaled, train_hist_scaled, train=False)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:45<00:00, 44.21it/s]


In [222]:
print("Train Matrix shape:", train_matrix.shape)
print("Test Matrix shape:", test_matrix.shape)

Train Matrix shape: (5000, 5000)
Test Matrix shape: (2000, 5000)


Cand spunem kernel precomputed, suntem responsabili sa construim o matrice de similaritate intre toate exemplele, in loc sa dam direct X_train cu features. 

Adica o matrice care contine:

$$ K_{i, j} = K(x_i, x_j) $$, unde $x_i$ si $x_j$ sunt exemple de input.

kernel = 'precomputed' ii spune lui SVM ca deja ii dam matricea de kernel, fara sa mai foloseasca 'linear', 'rbf' etc

SVC asteapta la antrenare un $(train\_matrix, y_{train})$:

- train_matrix trebuie sa aiba shape-ul $= (n_{train}, n_{train})$
    - Fiecare rand "i" e imaginea "i: din train, iar fiecare coloana "j" e imaginea "j" din train
 
$$ train\_matrix[i][j] = kernel(x_i, x_j) $$

La testare:

- test_matric trebuie sa aiba sahape-ul $(n_{test}, n_{train})$
    - Fiecare rand e o imagine de test, iar fiecare coloana e o imagine de train

$$ test\_matrix[i][j] = kernel(x_{\text{test}_i},\ x_{\text{train}_j}) $$

Comparam fiecare de imagine de test cu TOATE imaginile din train, ca sa o clasificam.


In [ ]:
clf = SVC(C = 0.01, kernel='precomputed') # 

clf.fit(train_matrix, train_labels)

# Facem transpusa matricii kernel de test pentru ca SVM se asteapta la shape = (n_test, n_train) si noi avem invers
print("Kernel Intersection SVM Accuracy:", clf.score(test_matrix, test_labels)*100, "%")

Kernel Intersection SVM Accuracy: 81.15 %


## Final observations

#### Exercitiul 1 - Local Binary Pattern

E o metoda de descriere a texturii unei imagini. Pentru fiecare pixel, comparam vecinii din jur (de obicei 3x3) cu valoarea pixelului central. Daca vecinul >= pixelul central, asignam valoarea 1, altfel 0. Rezulta un vector binar care encodeaza textura locala. Dupa care, acest vector e transformat in valoare zecimala si se face histograma pe imagine.

La Exercitiul 1 am implementat o varianta personalizata de LBP: am implementat o fereastra custom dxd, am creat semnatura binara, dupa care am creat histograma globala.

- Buna pentru descrierea texturii locala
- Rapida si eficienta pt imagini simple (caractere, semne etc.)

#### Exercitiul 2 - Magnitudinea Gradientului

Gradientul unei imagini (2D) descrie cat de rapid se schimba intensitatea pixelilor. E practic un vector (Gx, Gy), unde Gx e variatia pe orizontala, iar Gy e variatia pe verticala.

La Exercitiul 2 am aplicat operatorul Sobel pentru a calcula componentele gradientului Gx si Gy, am extras doar zonele cele mai relevante (cu magnitudine mare), adica unde avem contururi si margini clare, si am pastrat cele mai semnificate 30 de regiuni (cele mai mari medii de gradient).

- Bun pentru evidentiere de margini, forme, contururi
- Eficient in recunoastere de obiecte
- Se concentreaza doar pe forme, nu pe culoare sau textura

#### Exercitiul 3 - Directia Gradientului + NMS

Directia gradientului e orientarea schimbarii de intensitate (pe oblica, pe verticala etc.)

NMS (Non-Maximum Suppression) retine doar pixelii de pe directia gradientului care sunt maximi local. Elimina valori redundante sau neclare. E folosit in special in Edge Detection (algoritmul Canny), Feature Selection, Object Detection etc. Scopul e sa rarefieze contururile, pastrand doar pixelii maximali in "fereastra" lor de 3 pixeli, nu sa distruga complet.

La Exercitiul 3, am calculat directia gradientului pentru fiecare pixel, am aplicat NMS ca sa pastram pixelii maximali, dupa care am aplatizat imaginea rezultata si am folosit-o ca feature vector pentru SVM.

- Filtrare fina a marginilor
- Ne da o reprezentare mai curata a imaginii/structural vorbind
- Mentine doar cele mai relevante contururi

#### Exercitiul 4 - Binary Signature pe regiuni + Hamming KNN

La Exercitiul 4, am impartit imaginea in patch-uri care nu se suprapun, si pentru fiecare patch am generat un vector binar pe baza mediei sau valorii centrale. Dupa care, am concatenat acesti vectori intr-un vector binar mare (lungime 784 pentru 49 patch-uri de 4x4) si am folosit KNN cu distanta Hamming, care a masurat cati biti difera intre doua imagini.

- Vectorii binari sunt eficienti ca memorie si rapizi la comparare
- Distanta Hamming e naturala pentru reprezentari de genul asta

#### Exercitiul 5 - Kernel de Intersectie intre Histograme

La Exercitiul 5, am comparat histograme generate de Exercitiul 1 intre ele. Am folosit kernelul de intersectie, care e o functie de similaritate ce masoara cat de mult se "suprapun" doua histograme. El e pozitiv definit, deci e valid ca si kernel pentru SVM.

In formula pentru kernel, daca xi = yi pentru toate i, atunci suma e maxima => similaritate mare.
Daca xi = 0 sau yi = 0 in multe locuri, suma scade => similaritate mica.

Deci, chiar daca kernelul de intersectie e doar o suma, el reflecta volumul comun dintre doua histograme (efectiv intersectia dintre ele)

- Bun pentru clasificare de imagini reprezentate prin histograme (LBF, SIFT, HOG etc.)
- Util cand valorile sunt frecvente sau conteaza distributia (nu pozitia exacta)